In [106]:
import pandas as pd
import numpy as np
from random import sample

In [66]:
def h(values):
    return sum([int(x) + 6 for x in values]) % 100000
    
def create_pairs(items):
    pairs = []
    items = [int(x) for x in items]
    items = sorted (items)
    for i in range(len(items)):
        for j in range(i+1, len(items)):
            pairs.append((items[i],items[j]))
    return pairs

In [65]:
retail = pd.read_csv('http://fimi.uantwerpen.be/data/retail.dat')
records = retail.values.tolist()
for i in range(len(records)):
    records[i] = records[i][0].split()

In [107]:
percentages = [0.2,0.4,0.6,0.8,1]
thresholds = [0.01, 0.02, 0.05]
samples = []
durations = []

# Loop through percentages and make random samples 
for percentage in percentages:
    samples.append(sample(records,int((percentage)*len(records))))

In [101]:
items = {}
hash_table = np.zeros(100000)
support_threshold = len(records)*0.01

for i in range(len(records)):
    for j in range(len(records[i])):
        if records[i][j] not in items:
            items[records[i][j]] = 0 
        items[records[i][j]] += 1
        pairs = create_pairs(records[i])
        for pair in pairs:
            hash_table[h(pair)] += 1

C1 = pd.DataFrame({
    'Itemset': items.keys(),
    'Support': items.values()
})
L1 = C1.loc[(C1['Support']) >= support_threshold]

bitmap = np.zeros(100000, dtype=bool)
for i in range(len(bitmap)):
    bitmap[i] = hash_table[i] >= support_threshold

arr = []
for i in L1['Itemset']:
    for j in L1['Itemset']:
        if i == j:
            pass
        else:
            for k in records:
                if i in k and j in k and [i,j,0] not in arr and [j,i,0] not in arr:
                    arr.append([i,j,0])

for i in range(len(arr)):
            for j in records:
                if arr[i][0] in j and arr[i][1] in j and bitmap[h((arr[i][0],arr[i][1]))]:
                    arr[i][2] += 1

C2 = pd.DataFrame(arr, columns=['Item 1','Item 2','Count'])
L2 = C2.loc[(C2['Count']) >= support_threshold]

        